In [ ]:
import requests
import time
import json
from typing import Dict, List, Tuple, Optional


def authenticate(username: str, password: str) -> str:
   """
   Authenticate with the AppEEARS API and return the authentication token.


   :param username: NASA Earthdata Login username.
   :param password: NASA Earthdata Login password.
   :return: Authentication token.
   """
   response = requests.post(
       url='https://appeears.earthdatacloud.nasa.gov/api/login',
       auth=(username, password)
   )
   return response.json()['token']


def submit_task(token: str, task_params: Dict) -> str:
   """
   Submit a task to the AppEEARS API and return the task ID.


   :param token: Authentication token.
   :param task_params: Parameters for the task.
   :return: Task ID.
   """
   response = requests.post(
       url='https://appeears.earthdatacloud.nasa.gov/api/task',
       json=task_params,
       headers={'Authorization': f'Bearer {token}'}
   )
   return response.json()['task_id']


def check_task_status(token: str, task_id: str) -> Dict:
   """
   Check the status of a task.


   :param token: Authentication token.
   :param task_id: ID of the task to check.
   :return: Status of the task.
   """
   response = requests.get(
       url=f'https://appeears.earthdatacloud.nasa.gov/api/task/{task_id}',
       headers={'Authorization': f'Bearer {token}'}
   )
   return response.json()


def create_task(long: float, lat: float, start_date: str, end_date: str, product: str, file_type: str) -> Optional[Dict]:
   """
   Create a task with the AppEEARS API.


   :param long: Longitude.
   :param lat: Latitude.
   :param start_date: Start date of the period (format MM-DD-YYYY).
   :param end_date: End date of the period (format MM-DD-YYYY).
   :param product: Product to use.
   :param file_type: Type of data file (geotiff or nc4).
   :return: Task status if task is successfully completed, otherwise None.
   """
   # User credentials (replace with actual credentials)
   username = 'your-username'
   password = 'your-password'


   # Authenticate and get token
   token = authenticate(username=username, password=password)


   # Define task parameters
   task = {
       'task_type': 'point',
       'task_name': 'AppEEARS Data Request',
       'params': {
           'dates': [{'startDate': start_date, 'endDate': end_date}],
           'layers': [{'product': product, 'layer': 'LST_Day_1km'}],
           'coordinates': [{'latitude': lat, 'longitude': long}],
           'output': {'format': {'type': file_type}}
       }
   }


   # Submit the task
   task_id = submit_task(token=token, task_params=task)
   print(f"Task submitted, ID: {task_id}")


   # Check task status periodically
   while True:
       status = check_task_status(token=token, task_id=task_id)
       if status['status'] == 'done':
           print("Task completed successfully.")
           return status
       elif status['status'] == 'error':
           print("An error occurred.")
           return None
       else:
           print(f"Task status: {status['status']}. Waiting before next check...")
           time.sleep(60)  # Wait for 60 seconds before checking again


   return None